# Event Analytics
Performing analytics on events is a core feature TrendMiner, which makes interacting with events a core use case of the `trendminer_interface` package.

## Authentication
As always, as a first step we authenticate to the TrendMiner appliance.

In [1]:
import keyring
from trendminer_interface import TrendMinerClient

url = 'https://cs.trendminer.net/'
client_id = 'wdanielsclient'

client = TrendMinerClient(
    url = url,
    client_id = client_id,
    client_secret=keyring.get_password(url, client_id),
    tz = "Europe/Brussels"  # all returned timestamps will be in this timezone
)

/Users/wouter.daniels/PycharmProjects/tm-python-interface/trendminer_interface/client.py:191: VersionMismatchWarning: This SDK version was tested for use with TrendMiner versions [2024.R1.0 | 2024.R1.1 | 2024.R2.0 | 2024.R2.1] while your TrendMiner version is [2025.R1.0-04]. Some functionality might not work as expected. 
  warnings.warn(


## Selecting tags
We will need some demo tags to illustrate event analytics. We can load these directly from their names. 

In [2]:
flow = client.tag.from_name("TM4-HEX-FI0620")
temp = client.tag.from_name("TM4-HEX-TI0620")

## Selecting intervals
This package uses pandas [Interval](https://pandas.pydata.org/docs/reference/api/pandas.Interval.html) and [IntervalIndex](https://pandas.pydata.org/docs/reference/api/pandas.IntervalIndex.html#pandas.IntervalIndex) as central objects for denoting events and as method inputs.

Importing from `trendminer_interface` adds [custom accessors](https://pandas.pydata.org/docs/development/extending.html#registering-custom-accessors) to the DataFrame class, which contain TrendMiner-specific methods as well as some utility methods.

A common starting point is therefore to generate an IntervalIndex with a regular frequency, and use that to create a new DataFrame. Here, we select all full weeks (starting Monday) in April 2023.

In [3]:
import pandas as pd

week_index = pd.interval_range(
    start=pd.Timestamp("2023-04-01 00:00:00", tz=client.tz),
    end=pd.Timestamp("2023-05-01 00:00:00", tz=client.tz),
    freq="W-MON",
    closed="left",
)

df = pd.DataFrame(index=week_index)

## Calculations on intervals
We can now perform calculations usingt the custom `interval` accessor on the DataFrame. Accessor methods will return the manipulated DataFrame.

In [4]:
df.interval.calculate(tag=temp, operation="max", name="max temp")

max temp
[2023-04-03 00:00:00+02:00, 2023-04-10 00:00:00...  52.869370
[2023-04-10 00:00:00+02:00, 2023-04-17 00:00:00...  51.089703
[2023-04-17 00:00:00+02:00, 2023-04-24 00:00:00...  50.764526
[2023-04-24 00:00:00+02:00, 2023-05-01 00:00:00...  50.890480

To add multiple calculations, we can pipe DataFrame operations. We can of course include regular Python operations on the DataFrame. We can integrate the weekly flow to get the total volume, but because the TrendMiner integral assumes a time unit of 'day', and our flow is expressed in m3 per hour, we need to multiply by 24 to get the volume in m3. Then as an example, let's set some bound on the flow derived from the mean and standard deviation, keeping into account that the lower limit should never be lower than 0.

In [5]:
df = (
    pd.DataFrame(index=week_index)
    .interval.calculate(tag=flow, operation="int", name="total")
    .assign(volume=lambda df: df["total"]*24)
    .interval.calculate(tag=flow, operation="mean", name="mean")
    .interval.calculate(tag=flow, operation="std", name="std")
    .assign(upper_limit=lambda df: df["mean"]+1.5*df["std"])
    .assign(lower_limit=lambda df: (df["mean"]-1.5*df["std"]).clip(lower=0))
    .round(1)
)

df

total  volume  mean   std  \
[2023-04-03 00:00:00+02:00, 2023-04-10 00:00:00...  171.7  4121.8  24.5  17.5   
[2023-04-10 00:00:00+02:00, 2023-04-17 00:00:00...  338.3  8118.0  48.3  26.9   
[2023-04-17 00:00:00+02:00, 2023-04-24 00:00:00...  360.9  8661.6  51.6  29.0   
[2023-04-24 00:00:00+02:00, 2023-05-01 00:00:00...  300.9  7222.0  43.0  21.7   

                                                    upper_limit  lower_limit  
[2023-04-03 00:00:00+02:00, 2023-04-10 00:00:00...         50.8          0.0  
[2023-04-10 00:00:00+02:00, 2023-04-17 00:00:00...         88.7          8.0  
[2023-04-17 00:00:00+02:00, 2023-04-24 00:00:00...         95.0          8.1  
[2023-04-24 00:00:00+02:00, 2023-05-01 00:00:00...         75.5         10.5

## Interval manipulations
Various methods are available on the `interval` accessor to manipulate the timeframe given by events. Note that these manipulations only affect the IntervalIndex, and that DataFrame value columns are retained. For a full overview of methods, call `help(df.interval)`.

In [6]:
df.interval.shrink(left="1h", right="2h")

total  volume  mean   std  \
[2023-04-03 01:00:00+02:00, 2023-04-09 22:00:00...  171.7  4121.8  24.5  17.5   
[2023-04-10 01:00:00+02:00, 2023-04-16 22:00:00...  338.3  8118.0  48.3  26.9   
[2023-04-17 01:00:00+02:00, 2023-04-23 22:00:00...  360.9  8661.6  51.6  29.0   
[2023-04-24 01:00:00+02:00, 2023-04-30 22:00:00...  300.9  7222.0  43.0  21.7   

                                                    upper_limit  lower_limit  
[2023-04-03 01:00:00+02:00, 2023-04-09 22:00:00...         50.8          0.0  
[2023-04-10 01:00:00+02:00, 2023-04-16 22:00:00...         88.7          8.0  
[2023-04-17 01:00:00+02:00, 2023-04-23 22:00:00...         95.0          8.1  
[2023-04-24 01:00:00+02:00, 2023-04-30 22:00:00...         75.5         10.5

## Searches
Rather than selecting intervals manually, we can also retrieve them from a search. As an example, let's perform a value-based search on our flow being higher than 0. First we need to instantiate the search:

In [7]:
vbs = client.search.value(
    name="high flow",  # search name -> will become DataFrame index name
    queries=[
        (flow, ">", 60),  # our search query as a tuple (tag, operator, value)
    ],
    operator="AND",  # how to combine multiple queries (AND/OR)
    duration=pd.Timedelta(hours=24),  # minimal result duration
)

vbs

<< ValueBasedSearch | high flow >>

Our search needs to be performed on a specific interval to yield results as a dataframe. We use the pandas Interval class for this. We always need to assign a timezones to the timestamps used to generate the interval.

In [8]:
search_interval = pd.Interval(
    left=pd.Timestamp("2023-01-01", tz=client.tz),
    right=pd.Timestamp("2024-01-01", tz=client.tz),
    closed="both"
)

search_interval

Interval(2023-01-01 00:00:00+01:00, 2024-01-01 00:00:00+01:00, closed='both')

Performing the search yields an empty DataFrame with the search result intervals as the IntervalIndex

In [9]:
vbs.get_results(search_interval)

Empty DataFrame
Columns: []
Index: [[2023-04-16 06:00:00+02:00, 2023-04-18 21:20:00+02:00], [2023-04-22 13:32:00+02:00, 2023-04-23 15:55:00+02:00], [2023-04-26 19:25:00+02:00, 2023-04-28 04:02:00+02:00], [2023-05-06 16:40:00+02:00, 2023-05-08 21:01:00+02:00], [2023-05-08 21:44:00+02:00, 2023-05-10 04:40:00+02:00], [2023-05-13 11:29:00+02:00, 2023-05-15 04:38:00+02:00], [2023-05-20 15:43:00+02:00, 2023-05-21 20:02:00+02:00], [2023-05-27 19:39:00+02:00, 2023-06-01 08:26:00+02:00]]

To add calculations to the DataFrame, we can perform interval calculations as demonstrated in the previous section. Another option for searches is to include calculations already in the search definition:

In [10]:
vbs = client.search.value(
    name="high flow",
    queries=[
        (flow, ">", 60)
    ],
    operator="AND",
    duration=pd.Timedelta(hours=24),
    calculations={
        "max temp": (temp, "max"),
    },
)

df = vbs.get_results(search_interval)

df.head(3)

max temp
high flow                                                    
[2023-04-16 06:00:00+02:00, 2023-04-18 21:20:00...  50.742350
[2023-04-22 13:32:00+02:00, 2023-04-23 15:55:00...  50.764526
[2023-04-26 19:25:00+02:00, 2023-04-28 04:02:00...  50.890480

## Writing intervals to a tag
We can create tags from intervals using the csv import endpoint. First we need to instantiate a new tag object.

In [11]:
tag = client.tag(
    name=f"Example discrete calc tag",
    description="max batch temperature",
    tag_type="DISCRETE",
)

We also need a pandas Series with the tag data. This is easily generated from the DataFrame of calculations.

In [12]:
ser = pd.Series(index=df.index.left, data=df["max temp"], name=tag.name)
ser.head()

2023-04-16 06:00:00+02:00    50.742350
2023-04-22 13:32:00+02:00    50.764526
2023-04-26 19:25:00+02:00    50.890480
2023-05-06 16:40:00+02:00    50.206960
2023-05-08 21:44:00+02:00    51.009150
Name: Example discrete calc tag, dtype: float64

Then we can save/update this csv tag on the server

In [13]:
tag_data_dict = {tag: ser}
client.io.tag.save(tag_data_dict)

## Context items
Context items are basically intervals of a specific type, attached to a specific component. Let's take our search results from the previous section and turn them into context items.

Context items always have a specific `type`.

In [14]:
context_type = client.context.type.from_key("ANOMALY")
df["type"] = context_type

Context items must be attached to a `component`, which is either a tag, asset or attribute.

In [15]:
df["component"] = flow

Optionally, we can add a `description` to our context items as well.

In [16]:
df["description"] = "high flow"

We can now use the `context` accessor on the DataFrame. Let's use it to save our intervals as context items. Additional text or numeric columns (e.g., the `max temp` column) will be added as fields to the context items. If the column name is not mapped to a dedicated context field key on the context type, it will show up as 'other property' in TrendMiner.

In [17]:
df.context.save()

Saving the context items does not generate any output or updates the DataFrame, but we can retrieve our newly created context items through a ContextHub view. We see that saving the intervals as context items adds a lot of context item metadata to the intervals. 

For an overview of all possible filters on context items searches, call `help(client.context.filter)`.

In [18]:
chv = client.context.view(
    name="high flow",
    filters=[
        client.context.filter.components([flow]),
        client.context.filter.context_types([context_type]),
        client.context.filter.users([client.username]),
    ]
)

chv.get_items().head(3)

key  \
high flow                                                    
[2023-04-16 06:00:00+02:00, 2023-04-18 21:20:00...  FB9-05   
[2023-04-22 13:32:00+02:00, 2023-04-23 15:55:00...  FB9-06   
[2023-04-26 19:25:00+02:00, 2023-04-28 04:02:00...  FB9-07   

                                                                              identifier  \
high flow                                                                                  
[2023-04-16 06:00:00+02:00, 2023-04-18 21:20:00...  b9b9b3d0-5b8e-408b-beac-1f2e376cc855   
[2023-04-22 13:32:00+02:00, 2023-04-23 15:55:00...  14740c70-23ce-41fd-97b1-12851041ce18   
[2023-04-26 19:25:00+02:00, 2023-04-28 04:02:00...  c6d0fa2e-065a-43ea-9410-3a24cebaac20   

                                                   identifier_external  \
high flow                                                                
[2023-04-16 06:00:00+02:00, 2023-04-18 21:20:00...                None   
[2023-04-22 13:32:00+02:00, 2023-04-23 15:55:00...                None   
[2023-04-26 19:25:00+02:00, 2023-04-28 04:02:00...                None   

                                                   description     type  \
high flow                                                                 
[2023-04-16 06:00:00+02:00, 2023-04-18 21:20:00...   high flow  ANOMALY   
[2023-04-22 13:32:00+02:00, 2023-04-23 15:55:00...   high flow  ANOMALY   
[2023-04-26 19:25:00+02:00, 2023-04-28 04:02:00...   high flow  ANOMALY   

                                                         component  \
high flow                                                            
[2023-04-16 06:00:00+02:00, 2023-04-18 21:20:00...  TM4-HEX-FI0620   
[2023-04-22 13:32:00+02:00, 2023-04-23 15:55:00...  TM4-HEX-FI0620   
[2023-04-26 19:25:00+02:00, 2023-04-28 04:02:00...  TM4-HEX-FI0620   

                                                                        created_by  \
high flow                                                                            
[2023-04-16 06:00:00+02:00, 2023-04-18 21:20:00...  service-account-wdanielsclient   
[2023-04-22 13:32:00+02:00, 2023-04-23 15:55:00...  service-account-wdanielsclient   
[2023-04-26 19:25:00+02:00, 2023-04-28 04:02:00...  service-account-wdanielsclient   

                                                                            created  \
high flow                                                                             
[2023-04-16 06:00:00+02:00, 2023-04-18 21:20:00... 2025-05-06 12:59:30.570522+02:00   
[2023-04-22 13:32:00+02:00, 2023-04-23 15:55:00... 2025-05-06 12:59:30.570658+02:00   
[2023-04-26 19:25:00+02:00, 2023-04-28 04:02:00... 2025-05-06 12:59:30.570716+02:00   

                                                                      last_modified  \
high flow                                                                             
[2023-04-16 06:00:00+02:00, 2023-04-18 21:20:00... 2025-05-06 12:59:30.570522+02:00   
[2023-04-22 13:32:00+02:00, 2023-04-23 15:55:00... 2025-05-06 12:59:30.570658+02:00   
[2023-04-26 19:25:00+02:00, 2023-04-28 04:02:00... 2025-05-06 12:59:30.570716+02:00   

                                                     max temp MaterialName  \
high flow                                                                    
[2023-04-16 06:00:00+02:00, 2023-04-18 21:20:00...  50.742350         <NA>   
[2023-04-22 13:32:00+02:00, 2023-04-23 15:55:00...  50.764526         <NA>   
[2023-04-26 19:25:00+02:00, 2023-04-28 04:02:00...  50.890480         <NA>   

                                                   Suggestion2_psd  \
high flow                                                            
[2023-04-16 06:00:00+02:00, 2023-04-18 21:20:00...            <NA>   
[2023-04-22 13:32:00+02:00, 2023-04-23 15:55:00...            <NA>   
[2023-04-26 19:25:00+02:00, 2023-04-28 04:02:00...            <NA>   

                                                   suggestion1_psd  
high flow                                                           
[2

It is possible to update context items we have retrieved via a ContextHub view. We can simply manipulate the field columns and send an update request. A common use case is to enrich context items with additional calculations in a script. These context items could be created automatically from a monitor in TrendMiner.

A standard workflow here would be to retrieve context items where a specific field is empty, fill in this (and potentially other) fields in the DataFrame, and then update the items with the new data in TrendMiner.

Including a filter to only retrieve (and thus updated) items which have not yet been updated is essential for good performance. It also means our script can run at any time to update all new items.

In [19]:
material_key = "MaterialName"
material_field = client.context.field.from_key(material_key)

# Retrieve items with emtpy material field 
chv = client.context.view(
    name="high flow",
    filters=[
        client.context.filter.components([flow]),
        client.context.filter.context_types([context_type]),
        client.context.filter.users([client.username]),
        client.context.filter.field(material_field, mode="EMPTY"),
    ]
)
df = chv.get_items()

# Calculate the maximal flow
df = df.interval.calculate(flow, "max", "max flow")

# Determine if it was product A or B, depending on wether the max flow was above 100
# We use the `pop` method to remove fields we do not want to add to the context items.
df["is A"] = df.pop("max flow") > 100
df[material_key] = df.pop("is A").map({True: "A", False: "B"})

# Update the context items on the server
df.context.update()

Finally, it is also possible to delete context items directly from a ContextHub view. Including a filter on the user is recommended so we do not accidentally delete other people's context items.

In [20]:
chv = client.context.view(
    name="high flow",
    filters=[
        client.context.filter.components([flow]),
        client.context.filter.context_types([context_type]),
        client.context.filter.users([client.username]),
    ]
)

chv.delete_items()